# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3)

#### Parte 4: Evaluación de resultados.

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

---
# Datasets

## Carga

Los datasets que necesitamos son:
* outliers >> información sobre los outliers encontrados en cada subrubro mediante DBSCAN.
* sr10 >> dataset pivoteado con los 10 subrubros que quedaron, luego de imputar con KNN y hacer PCA de 12 componentes.
* pivot >> dataset pivoteado completo.
* orig >> dataset curado, previo al punto 11 (ya normalizados por inflación).

In [2]:
outliers = pd.read_csv('../data/interim/tp3_outliers_dbscan.csv')

sr10 = pd.read_csv('../data/interim/tp3_sr10_knn5_pca12.csv')

pivot = pd.read_csv('../data/interim/tp3_vendedores_vector.csv')
pivot = pivot.drop(columns='Omega').copy()

orig = pd.read_csv('../data/interim/tp3_ventas_antesdel11.csv')
orig = orig.drop(columns='Omega').copy()

## Uniformando según subrubros

Sabemos que de todos los subrubros, sólo vamos a poder responder a 9 de ellos (descontando `Tabaco`). Estos son:
* `Com. Varios`
* `Comb.`
* `Comb. Ley`
* `Farmacia`
* `Gondola`
* `Miscelaneo`
* `Supermercados`
* `Vehiculos`
* `Venta Agrop.`

Vemos primero qué subrubros hay en cada dataset para luego filtrarlos según este criterio.

In [3]:
print(f'\noutliers: {len(outliers["Subrubro"].unique())}')
print(outliers["Subrubro"].unique())

print(f'\nsr10: {len(sr10["Subrubro"].unique())}')
print(sr10["Subrubro"].unique())

print(f'\npivot: {len(pivot["Subrubro"].unique())}')
print(pivot["Subrubro"].unique())

print(f'\norig: {len(orig["Subrubro"].unique())}')
print(orig["Subrubro"].unique())

# Quitamos 'Tabaco'
outliers = outliers[outliers["Subrubro"] != 'Tabaco'].copy()
sr10 = sr10[sr10["Subrubro"] != 'Tabaco'].copy()
pivot = pivot[pivot["Subrubro"] != 'Tabaco'].copy()
orig = orig[orig["Subrubro"] != 'Tabaco'].copy()

# Quitamos 'Comb. Reventa'
pivot = pivot[pivot["Subrubro"] != 'Comb. Reventa'].copy()
orig = orig[orig["Subrubro"] != 'Comb. Reventa'].copy()


outliers: 10
['Com. Varios' 'Comb.' 'Comb. Ley' 'Farmacia' 'Gondola' 'Miscelaneo'
 'Supermercados' 'Tabaco' 'Vehiculos' 'Venta Agrop.']

sr10: 10
['Com. Varios' 'Comb.' 'Comb. Ley' 'Farmacia' 'Gondola' 'Miscelaneo'
 'Supermercados' 'Tabaco' 'Vehiculos' 'Venta Agrop.']

pivot: 11
['Miscelaneo' 'Gondola' 'Com. Varios' 'Venta Agrop.' 'Supermercados'
 'Farmacia' 'Comb.' 'Comb. Ley' 'Vehiculos' 'Tabaco' 'Comb. Reventa']

orig: 11
['Venta Agrop.' 'Vehiculos' 'Com. Varios' 'Tabaco' 'Gondola' 'Miscelaneo'
 'Comb.' 'Comb. Ley' 'Farmacia' 'Supermercados' 'Comb. Reventa']


## Uniformando según ID vendedor por subrubro

El conjunto de valores para `ID` en el dataset `outliers` ya es un subconjunto de `sr10`. Necesitamos que los datasets `pivot` y `orig` tengan los mismos `ID` que `sr10`. 

In [4]:
print(f'\nsr10: {len(sr10["ID"].unique())}')
print(f'\npivot: {len(pivot["ID"].unique())}')
print(f'\norig: {len(orig["ID"].unique())}')

id_sr10 = sr10["ID"].sort_values().unique()
id_pivot = pivot["ID"].sort_values().unique()
id_orig = orig["ID"].sort_values().unique()

drop_id_orig = []
for id in id_orig:
    if id not in id_sr10:
        drop_id_orig.append(id)
    
drop_id_orig = np.array(drop_id_orig)

drop_id_pivot = []
for id in id_pivot:
    if id not in id_sr10:
        drop_id_pivot.append(id)
    
drop_id_pivot = np.array(drop_id_pivot)


sr10: 2324

pivot: 2799

orig: 3137


In [5]:
for id in drop_id_pivot:
    pivot = pivot[pivot["ID"] != id].copy()

print(f'\npivot: {len(pivot["ID"].unique())}')

for id in drop_id_orig:
    orig = orig[orig["ID"] != id].copy()

print(f'\norig: {len(orig["ID"].unique())}')


pivot: 2324

orig: 2324
